In [110]:
import mysql.connector

class Blast:
    def __init__(self, window):
        self.window = window
        self.cnx = SQLC.connect(
            host ="geo.di.uminho.pt",
            user ="bioinformatica",
            password ="20221207",
            database ="AP_db_KRG"
        )
    def search_db(self, query):
        cursor = self.cnx.cursor()
        cursor.execute("SELECT sequence FROM Gene WHERE sequence LIKE %s", ('%'+query+'%',))
        sequences = cursor.fetchall()
        cursor.close()
        if sequences:
            self.sequence = sequences[0][0]
            return True
        else:
            return False

    def hits(self, query_map, sequence):
        hits = []
        for i in range(len(sequence) - self.window):
            window = sequence[i:i+self.window]
            score = 0
            for q in query_map:
                if window.count(q) > 0:
                    score += 1
            if score == len(query_map):
                hits.append((i, window))
        return hits

    def query_map(self, query):
        return set(query)

    def run_blast(self, query):
        if self.search_db(query):
            qm = self.query_map(query)
            all_hits = self.hits(qm, self.sequence)
            return all_hits
        else:
            return "No results found."




In [111]:
# Usage example
blast = Blast(20)
query = input("Enter a sequence: ")
results = blast.run_blast(query)
print(results)

Enter a sequence: TACCTT
[(0, 'TACCTTGTCC...CCAAAGG'), (1, 'ACCTTGTCC...CCAAAGGG'), (2, 'CCTTGTCC...CCAAAGGGC')]


In [135]:
class Blast:
    def __init__(self, w):
        self.w = w
        self.cnx = SQLC.connect(
            host ="geo.di.uminho.pt",
            user ="bioinformatica",
            password ="20221207",
            database ="AP_db_KRG"
        )

    def search_db(self, query):
        cursor = self.cnx.cursor()
        cursor.execute("SELECT sequence FROM Gene WHERE sequence LIKE %s", ('%'+query+'%',))
        sequences = cursor.fetchall()
        cursor.close()
        if sequences:
            self.sequences = sequences
            return True
        else:
            return False

    def query_map(self, query):
        tam = len(query)
        res = {}
        for chave, offset in [(query[p: p + self.w], p) for p in range(0, tam - self.w + 1)]:
            if chave not in res: res[chave] = []
            res[chave].append(offset)
        return res

    def get_all_offsets(self, s1, s2):
        w = len(s1)
        res = []
        for p in range(0, len(s2) - w + 1):
            if s2[p : p + w] == s1:
                res.append(p)
        return res

    def hits(self, query_map, sequences):
        res = []
        for chave, offsets in query_map.items():
            for o_query in offsets:
                for o_seq in self.get_all_offsets(chave, sequences):
                    res.append((o_query, o_seq))
        return res

    def extend_hit_dir(self, query, seq, o1, o2, direction):
        matches = 0
        count = 0
        while o1 >= 0 and o2 >= 0 and o1 < len(query) and o2 < len(seq):
            matches += 1 if query[o1] == seq[o2] else 0
            count += 1
            if 2 * matches < count:
                return o1, o2, matches, count - 1
            o1 += direction
            o2 += direction
        return o1 - direction, o2 - direction, matches, count

    def extend_hit(self, query, seq, hit, w):
        o1, o2 = hit
        left  = self.extend_hit_dir(query, seq, o1 - 1, o2 - 1, -1)
        right = self.extend_hit_dir(query, seq, o1 + w, o2 + w, +1)

        O1, O2, ML, SL = left
        _,   _, MR, SR = right

        return O1, O2, w + SL + SR, ML + w + MR

    def run_blast(self, query):
        if self.search_db(query):
            query_map = self.query_map(query)
            all_hits = self.hits(query_map, self.sequences)
            final_hits = []
            for hit in all_hits:
                extend_hit_res = self.extend_hit(query, self.sequences, hit, self.w)
                final_hits.append((query, hit[0], hit[1], extend_hit_res[2], extend_hit_res[3]))
            return final_hits
        else:
            return "No results found."


In [136]:
blast = Blast(15)
query = input("Enter a sequence: ")
results = blast.run_blast(query)
print(results)

Enter a sequence: CCAAAGGGCC
[]


In [154]:
class Blast:
    def __init__(self, w):
        self.w = w
        self.cnx = SQLC.connect(
            host ="geo.di.uminho.pt",
            user ="bioinformatica",
            password ="20221207",
            database ="AP_db_KRG"
        )
    def search_db(self, query):
        cursor = self.cnx.cursor()
        cursor.execute("SELECT sequence FROM Gene WHERE sequence LIKE %s", ('%'+query+'%',))
        sequences = cursor.fetchall()
        cursor.close()
        return sequences
        
    def query_map(self, query):
        tam = len(query)
        res = {}
        for chave, offset in [(query[p: p + self.w], p) for p in range(0, tam - self.w + 1)]:
            if chave not in res: res[chave] = []
            res[chave].append(offset)
        return res
        
    def get_all_offsets(self, s1, s2):
        w = len(s1)
        res = []
        for p in range(0, len(s2) - w + 1):
            if s2[p : p + w] == s1:
                res.append(p)
        return res
    
    def hits(self, query_map, sequence, w):
        res = []
        for chave, offsets in query_map.items():
            for o_query in offsets:
                for o_seq in self.get_all_offsets(chave, sequence):
                    res.append((o_query, o_seq))
        return res

    def run_blast(self, query):
        sequences = self.search_db(query)
        print(sequences)
        if sequences:
            qm = self.query_map(query)
            all_hits = []
            for sequence in sequences:
                all_hits.extend(self.hits(qm, sequence, self.w))
            return all_hits
        else:
            return "No results found."


In [156]:
blast = Blast(15)
query = input("Enter a sequence: ")
results = blast.run_blast(query)
print(results)

Enter a sequence: TACCTT
[('TACCTTGTCC...CCAAAGGGCC',)]
[]


In [150]:
def search_db(query):
    cnx = SQLC.connect(
            host ="geo.di.uminho.pt",
            user ="bioinformatica",
            password ="20221207",
            database ="AP_db_KRG")
    cursor = cnx.cursor()
    cursor.execute("SELECT sequence FROM Gene WHERE sequence LIKE %s", ('%'+query+'%',))
    sequences = cursor.fetchall()
    print(sequences)
    cursor.close()
    return sequences
search_db("AA")

[('TACCTTGTCC...CCAAAGGGCC',), ('CAACACGGGA...AACCTCACCC',), ('GGGATGGAAG...TGGCGGCCTA',), ('GTTTGGTGAA...CTTCTGCCTA',), ('CCAAGAACAA...GCCTCTGACA',), ('TGTGAATGAA...GTTTGGCCGT',), ('TTTTTTAAGT...GGGTTCATCC',), ('GAATTCGTGC...CGGGGAATTC',), ('GGAAGTAAAA...AAGCGGAGGA',)]


[('TACCTTGTCC...CCAAAGGGCC',),
 ('CAACACGGGA...AACCTCACCC',),
 ('GGGATGGAAG...TGGCGGCCTA',),
 ('GTTTGGTGAA...CTTCTGCCTA',),
 ('CCAAGAACAA...GCCTCTGACA',),
 ('TGTGAATGAA...GTTTGGCCGT',),
 ('TTTTTTAAGT...GGGTTCATCC',),
 ('GAATTCGTGC...CGGGGAATTC',),
 ('GGAAGTAAAA...AAGCGGAGGA',)]